# Lab 2.1: Average Friends by Age

## Tổng quan bài tập
**Đề bài**: Dựa vào tập dữ liệu, hãy tính số lượng bạn trung bình theo từng tuổi. Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán trên.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1br4OUyHIjffJj88vc6aofVr2fe0s4t1x/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file .csv gồm 3 cột theo thứ tự sau:
- `Index`: Số thứ tự.
- `Name`: Tên của người dùng.
- `Age`: Tuổi của người đó.
- `Friends`: Số lượng bạn hiện tại của người đó.

Ngoài ra, bạn có thể tham khảo các video sau trong trường hợp chưa hiểu cách làm bài Lab:
- [Average Friends by Age Example](https://fpt-software.udemy.com/course/taming-big-data-with-apache-spark-hands-on/learn/lecture/3710440#overview)


# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
38 packages can be upgraded. Run 'apt list --upgradable' to see them.


Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [3]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [5]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Average Friends by Age

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('avgAgeOfFriends')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

Đầu tiên, bạn sẽ cần đọc dữ liệu từ Dataset và lưu dưới dạng một RDD

In [8]:
DATASET_PATH = "/content/gdrive/My Drive/DEP303/fakefriends.csv"
lines = sc.textFile(DATASET_PATH)

Do được đọc từ file nên RDD đang ở dưới một danh sách các dòng trong file. Bạn cần chuyển RDD này về một danh sách các Key-Value với Key là tuổi và Value là số lượng bạn bè.
Ví dụ `0	Will	33	385` sẽ được chuyển thành `(33, 385)`


In [12]:
def parseLine(line):
    cells = line.split(',')
    age = int(cells[2])
    numFriends = int(cells[3])

    return (age, numFriends)


rdd = lines.map(parseLine)

Sau đó, bạn cần tổng hợp lại số người ở cùng một tuổi cũng như tổng số lượng bạn bè của những người đó, sau đó sẽ tính ra được số lượng bạn bè trung bình và đưa ra kết quả

In [13]:
totalsByAge = rdd \
                .mapValues(lambda x: (x, 1)) \
                .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

averagesByAge = totalsByAge.mapValues(lambda x: x[0] / x[1])
results = averagesByAge.collect()

for result in results:
    print(result)

(33, 325.3333333333333)
(26, 242.05882352941177)
(55, 295.53846153846155)
(40, 250.8235294117647)
(68, 269.6)
(59, 220.0)
(37, 249.33333333333334)
(54, 278.0769230769231)
(38, 193.53333333333333)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(56, 306.6666666666667)
(43, 230.57142857142858)
(36, 246.6)
(22, 206.42857142857142)
(35, 211.625)
(45, 309.53846153846155)
(60, 202.71428571428572)
(67, 214.625)
(19, 213.27272727272728)
(30, 235.8181818181818)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(42, 303.5)
(49, 184.66666666666666)
(48, 281.4)
(50, 254.6)
(39, 169.28571428571428)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(31, 267.25)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(41, 268.55555555555554)
(44, 282.1666666666667)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(29, 215.91666666666666)
(18, 343.375)
(47, 233.22222222222